In [1]:
# https://codeshare.io/pAe0RV
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=b3f121c3595d4f009275ab679fe29198a4f0e64c58d94b6e83614d570a24701e
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
# step 1
import pandas as pd
from datetime import datetime

data = {
    "TransactionID": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "CustomerID": [101, 102, 103, 101, 104, 102, 103, 104, 101, 105],
    "ProductID": [501, 502, 501, 503, 504, 502, 503, 504, 501, 505],
    "Quantity": [2, 1, 4, 3, 1, 2, 5, 1, 2, 1],
    "Price": [150.0, 250.0, 150.0, 300.0, 450.0, 250.0, 300.0, 450.0, 150.0, 550.0],
    "Date": [
        datetime(2024, 9, 1),
        datetime(2024, 9, 1),
        datetime(2024, 9, 2),
        datetime(2024, 9, 2),
        datetime(2024, 9, 3),
        datetime(2024, 9, 3),
        datetime(2024, 9, 4),
        datetime(2024, 9, 4),
        datetime(2024, 9, 5),
        datetime(2024, 9, 5)
    ]
}

# DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/DataEngineering/sales_data.csv', index=False)

print("Sample sales dataset has been created and saved as 'sales_data.csv'.")

Sample sales dataset has been created and saved as 'sales_data.csv'.


In [10]:
# step 2

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Sales Dataset Analysis").getOrCreate()

sales_df = spark.read.csv("/content/drive/MyDrive/DataEngineering/sales_data.csv", header=True, inferSchema=True)

sales_df.show(4) #verify

+-------------+----------+---------+--------+-----+----------+
|TransactionID|CustomerID|ProductID|Quantity|Price|      Date|
+-------------+----------+---------+--------+-----+----------+
|            1|       101|      501|       2|150.0|2024-09-01|
|            2|       102|      502|       1|250.0|2024-09-01|
|            3|       103|      501|       4|150.0|2024-09-02|
|            4|       101|      503|       3|300.0|2024-09-02|
+-------------+----------+---------+--------+-----+----------+
only showing top 4 rows



In [6]:
# step 3

# printing schema
sales_df.printSchema()

# first 5 rows
sales_df.show(5)

# summary statistics for numeric columns
sales_df.describe(["Quantity", "Price"]).show()

root
 |-- TransactionID: integer (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- ProductID: integer (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Price: double (nullable = true)
 |-- Date: date (nullable = true)

+-------------+----------+---------+--------+-----+----------+
|TransactionID|CustomerID|ProductID|Quantity|Price|      Date|
+-------------+----------+---------+--------+-----+----------+
|            1|       101|      501|       2|150.0|2024-09-01|
|            2|       102|      502|       1|250.0|2024-09-01|
|            3|       103|      501|       4|150.0|2024-09-02|
|            4|       101|      503|       3|300.0|2024-09-02|
|            5|       104|      504|       1|450.0|2024-09-03|
+-------------+----------+---------+--------+-----+----------+
only showing top 5 rows

+-------+-----------------+-----------------+
|summary|         Quantity|            Price|
+-------+-----------------+-----------------+
|  count|            

In [19]:
# step 4

# 1.Calculate the Total Sales Value for Each Transaction
sales_df = sales_df.withColumn("TotalSales", sales_df["Quantity"] * sales_df["Price"])
sales_df.show()


+-------------+----------+---------+--------+-----+----------+----------+
|TransactionID|CustomerID|ProductID|Quantity|Price|      Date|TotalSales|
+-------------+----------+---------+--------+-----+----------+----------+
|            1|       101|      501|       2|150.0|2024-09-01|     300.0|
|            2|       102|      502|       1|250.0|2024-09-01|     250.0|
|            3|       103|      501|       4|150.0|2024-09-02|     600.0|
|            4|       101|      503|       3|300.0|2024-09-02|     900.0|
|            5|       104|      504|       1|450.0|2024-09-03|     450.0|
|            6|       102|      502|       2|250.0|2024-09-03|     500.0|
|            7|       103|      503|       5|300.0|2024-09-04|    1500.0|
|            8|       104|      504|       1|450.0|2024-09-04|     450.0|
|            9|       101|      501|       2|150.0|2024-09-05|     300.0|
|           10|       105|      505|       1|550.0|2024-09-05|     550.0|
+-------------+----------+---------+--

In [20]:
# 2.Group By ProductID and Calculate Total Sales Per Product
total_sales_by_product = sales_df.groupBy("ProductID").sum("TotalSales").withColumnRenamed("sum(TotalSales)","TotalSales")\
                        .orderBy("sum(TotalSales)", ascending=False)
total_sales_by_product.show()

+---------+----------+
|ProductID|TotalSales|
+---------+----------+
|      503|    2400.0|
|      501|    1200.0|
|      504|     900.0|
|      502|     750.0|
|      505|     550.0|
+---------+----------+



In [21]:
# 3.Identify the Top-Selling Product
top_selling_product = total_sales_by_product.limit(1)
top_selling_product.show()

+---------+----------+
|ProductID|TotalSales|
+---------+----------+
|      503|    2400.0|
+---------+----------+



In [22]:
# 4.Calculate the Total Sales by Date
total_sales_by_date = sales_df.groupBy("Date").sum('TotalSales').orderBy("Date")
total_sales_by_date.show()



+----------+---------------+
|      Date|sum(TotalSales)|
+----------+---------------+
|2024-09-01|          550.0|
|2024-09-02|         1500.0|
|2024-09-03|          950.0|
|2024-09-04|         1950.0|
|2024-09-05|          850.0|
+----------+---------------+



In [23]:
# 5.Filter High-Value Transactions
high_value_transactions = sales_df.filter(sales_df["TotalSales"] > 500)
high_value_transactions.show()

+-------------+----------+---------+--------+-----+----------+----------+
|TransactionID|CustomerID|ProductID|Quantity|Price|      Date|TotalSales|
+-------------+----------+---------+--------+-----+----------+----------+
|            3|       103|      501|       4|150.0|2024-09-02|     600.0|
|            4|       101|      503|       3|300.0|2024-09-02|     900.0|
|            7|       103|      503|       5|300.0|2024-09-04|    1500.0|
|           10|       105|      505|       1|550.0|2024-09-05|     550.0|
+-------------+----------+---------+--------+-----+----------+----------+



In [24]:
# 6.Identify Repeat Customers
repeat_customers = sales_df.groupBy("CustomerID").count().filter("count > 1").orderBy("count", ascending=False)
repeat_customers.show()


+----------+-----+
|CustomerID|count|
+----------+-----+
|       101|    3|
|       103|    2|
|       102|    2|
|       104|    2|
+----------+-----+



In [26]:
# 7.Calculate the Average Sale Price Per Product
average_sale_price = sales_df.groupBy("ProductID").avg('Price').withColumnRenamed("avg(Price)","AveragePrice").orderBy("AveragePrice", ascending=False)
average_sale_price.show()

+---------+------------+
|ProductID|AveragePrice|
+---------+------------+
|      505|       550.0|
|      504|       450.0|
|      503|       300.0|
|      502|       250.0|
|      501|       150.0|
+---------+------------+



# Exercise: Analyzing a Sample Sales Dataset Using PySpark

In this exercise, you'll work with a simulated sales dataset and perform various data transformations and analyses using PySpark. The dataset includes fields like `TransactionID`, `CustomerID`, `ProductID`, `Quantity`, `Price`, and `Date`. Your task is to generate the dataset, load it into PySpark, and answer specific questions by performing data operations.

---

### **Part 1: Dataset Preparation**

#### **Step 1: Generate the Sample Sales Dataset**

Before starting the analysis, you'll need to create the sample sales dataset. Use the following Python code to generate the dataset and save it as a CSV file.

1. **Run the Dataset Preparation Script:**

   ```python
   import pandas as pd
   from datetime import datetime

   # Sample sales data
   data = {
       "TransactionID": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
       "CustomerID": [101, 102, 103, 101, 104, 102, 103, 104, 101, 105],
       "ProductID": [501, 502, 501, 503, 504, 502, 503, 504, 501, 505],
       "Quantity": [2, 1, 4, 3, 1, 2, 5, 1, 2, 1],
       "Price": [150.0, 250.0, 150.0, 300.0, 450.0, 250.0, 300.0, 450.0, 150.0, 550.0],
       "Date": [
           datetime(2024, 9, 1),
           datetime(2024, 9, 1),
           datetime(2024, 9, 2),
           datetime(2024, 9, 2),
           datetime(2024, 9, 3),
           datetime(2024, 9, 3),
           datetime(2024, 9, 4),
           datetime(2024, 9, 4),
           datetime(2024, 9, 5),
           datetime(2024, 9, 5)
       ]
   }

   # Create a DataFrame
   df = pd.DataFrame(data)

   # Save the DataFrame to a CSV file
   df.to_csv('sales_data.csv', index=False)

   print("Sample sales dataset has been created and saved as 'sales_data.csv'.")
   ```

2. **Verify the Dataset:**
   - After running the script, ensure that the file `sales_data.csv` has been created in your working directory.

---

### **Part 2: Load and Analyze the Dataset Using PySpark**

Now that you have the dataset, your task is to load it into PySpark and perform the following analysis tasks.

#### **Step 2: Load the Dataset into PySpark**

1. **Initialize the SparkSession:**
   - Create a Spark session named `"Sales Dataset Analysis"`.

2. **Load the CSV File into a PySpark DataFrame:**
   - Load the `sales_data.csv` file into a PySpark DataFrame.
   - Display the first few rows of the DataFrame to verify that the data is loaded correctly.

#### **Step 3: Explore the Data**

Explore the data to understand its structure.

1. **Print the Schema:**
   - Display the schema of the DataFrame to understand the data types.

2. **Show the First Few Rows:**
   - Display the first 5 rows of the DataFrame.

3. **Get Summary Statistics:**
   - Get summary statistics for numeric columns (`Quantity` and `Price`).

#### **Step 4: Perform Data Transformations and Analysis**

Perform the following tasks to analyze the data:

1. **Calculate the Total Sales Value for Each Transaction:**
   - Add a new column called `TotalSales`, calculated by multiplying `Quantity` by `Price`.

2. **Group By ProductID and Calculate Total Sales Per Product:**
   - Group the data by `ProductID` and calculate the total sales for each product.

3. **Identify the Top-Selling Product:**
   - Find the product that generated the highest total sales.

4. **Calculate the Total Sales by Date:**
   - Group the data by `Date` and calculate the total sales for each day.

5. **Filter High-Value Transactions:**
   - Filter the transactions to show only those where the total sales value is greater than ₹500.

---

### **Additional Challenge (Optional):**

If you complete the tasks above, try extending your analysis with the following challenges:

1. **Identify Repeat Customers:**
   - Count how many times each customer has made a purchase and display the customers who have made more than one purchase.

2. **Calculate the Average Sale Price Per Product:**
   - Calculate the average price per unit for each product and display the results.